# Exercises 5: Using Kanren to Solve Logic Problems

## 

Use Kanren to solve the following logic puzzle:

Anjali, Brian, and Chen graduated from QUB on different days this summer. One of them studied French, one studied Geography, one read History. One scored a good 60%, one acheived a very good 65%, and the other an excellent 70%. 

We have acquired some knowledge about who studied what and what their marks were. However, our knowledge is incomplete. Here is what we know:

1. The person who studied French scored 65%.
2. Anjali studied Geography.
3. Brian's mark was 60%.

Who studied what, and what were their marks?

In [51]:
from kanren import run, var, membero, eq, lall, lany

graduations = var()

rules = lall(
    (eq, (var(), var(), var()), graduations),
    (membero, (var(), "French", "65%"), graduations),
    (membero, ("Anjali", "Geography", var()), graduations),
    (membero, ("Brian", var(), "60"), graduations),
    (membero, (var(), var(), "70"), graduations),
    (membero, ("Chen", var(), var()), graduations),
    (membero, (var(), "History", var()), graduations),
)

run(0, graduations, rules)

((('Chen', 'French', '65%'),
  ('Anjali', 'Geography', '70'),
  ('Brian', 'History', '60')),
 (('Anjali', 'Geography', '70'),
  ('Chen', 'French', '65%'),
  ('Brian', 'History', '60')),
 (('Anjali', 'Geography', '70'),
  ('Brian', 'History', '60'),
  ('Chen', 'French', '65%')),
 (('Chen', 'French', '65%'),
  ('Brian', 'History', '60'),
  ('Anjali', 'Geography', '70')),
 (('Brian', 'History', '60'),
  ('Chen', 'French', '65%'),
  ('Anjali', 'Geography', '70')),
 (('Brian', 'History', '60'),
  ('Anjali', 'Geography', '70'),
  ('Chen', 'French', '65%')))

## A Royal Flush

Given the following extract from a family tree, write functions that uses Kanren to

1. Find all siblings (brothers and sisters) of a given individual.
1. Find all cousins of a given individual (cousins have the same grandparents).
1. Find all ancestors of a given individual.


In [1]:
from kanren import Relation, facts
Parent = Relation()
facts(Parent, ("Elizabeth", "Charles"), ("Phillip", "Charles"), ("Charles", "William"), ("Charles", "Harry"), ("Diana", "William"), ("Diana", "Harry"), ("William","George"), ("Kate","George"), ("William","Charlotte"), ("Kate","Charlotte"), ("William","Louis"), ("Kate","Louis"), ("Harry","Archie"), ("Megan","Archie"), ("Harry","Lillibet"), ("Megan","Lillibet"), ("Elizabeth", "Anne"), ("Phillip", "Anne"), ("Anne","Peter"), ("Mark","Peter"), ("Anne","Zara"), ("Mark","Zara"), ("Peter","Savannah"), ("Autumn","Savannah"), ("Peter","Isla"), ("Autumn","Isla"), ("Zara","Mia"), ("Mike","Mia"), ("Zara","Lena"), ("Mike","Lena"), ("Zara","Lucas"), ("Mike","Lucas"), ("Elizabeth", "Andrew"), ("Phillip", "Andrew"), ("Andrew","Eugenie"), ("Sarah","Eugenie"), ("Andrew","Beatrice"), ("Sarah","Beatrice"), ("Beatrice","Sienna"), ("Edoardo","Sienna"), ("Eugenie","August"), ("Jack","August"), ("Eugenie","Ernest"), ("Jack","Ernest"), ("Elizabeth", "Edward"), ("Phillip", "Edward"), ("Edward","James"), ("Sophie","James"), ("Edward","Louise"), ("Sophie","Louise"))

In [14]:
from kanren import vars, run
def siblings(person):
    x, y = vars(2)
    siblings = set(run(0, x, Parent(y,x), Parent(y,person)))
    siblings.remove(person)
    return set(siblings)

siblings("Archie")

{'Lillibet'}

In [17]:
def cousins(person):
    w,x,y,z = vars(4)
    generation = set(run(0, w, Parent(x,w), Parent(z,x), Parent(z,y), Parent(y,person)))
    return set(generation) - set(siblings(person)) - set((person,))

cousins("Louis")

{'Archie', 'Lillibet'}

In [13]:
from kanren import var
def ancestors(person, lst = []):
    x = var()
    lst.append(person)
    parents = run(0, x, Parent(x,person))
    if len(parents) == 0:
        return None
    else:
        for p in parents:
            ancestors(p)
        return lst

print(ancestors("Lillibet"))

['Lillibet', 'Harry', 'Diana', 'Charles', 'Elizabeth', 'Phillip', 'Megan']


## Einstein's Riddle

Write a Kanren program to solve Einsteins riddle. There are five houses in a row. Each house is a different colour and is owned by a person of a different nationality. Each person keeps a different animal as a pet, prefers a different drink, and smokes a different brand of cigarette. Fifteen facts are known about the people and their houses:

1. There are five houses.
1. The Englishman lives in the red house.
1. The Spaniard owns the dog.
1. Coffee is drunk in the green house.
1. The Ukrainian drinks tea.
1. The green house is immediately to the right of the ivory house.
1. The Old Gold smoker owns snails.
1. Kools are smoked in the yellow house.
1. Milk is drunk in the middle house.
1. The Norwegian lives in the first house.
1. The man who smokes Chesterfields lives in the house next to the man with the fox.
1. Kools are smoked in the house next to the house where the horse is kept.
1. The Lucky Strike smoker drinks orange juice.
1. The Japanese smokes Parliaments.
1. The Norwegian lives next to the blue house.
 
Who drinks water? Who owns the zebra?

### Hints
* You will need to write "helper" functions to construct the goals corresponding to left/next to/right. You may find the `zip` function helpful.
* You can use the ordering of the solutions to represent the order of the houses.

In [25]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(*iterables) --> A zip object yielding tuples until an input is exhausted.
 |  
 |     >>> list(zip('abcdefg', range(3), range(4)))
 |     [('a', 0, 0), ('b', 1, 1), ('c', 2, 2)]
 |  
 |  The zip object yields n-length tuples, where n is the number of iterables
 |  passed as positional arguments to zip().  The i-th element in every tuple
 |  comes from the i-th iterable argument to zip().  This continues until the
 |  shortest argument is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and ret

In [ ]:
from kanren import var, lall, lany, membero, eq, run

def leftof(x, y, lst):
    return membero((x, y), zip(lst, lst[1:]))

def nextto(x,y,lst):
    return lany(leftof(x,y,lst),leftof(y,x,lst))

houses = var()

rules = lall(
    (eq, (var(), var(), var(), var(), var()), houses),
    (membero, ("Englishman", var(), var(), var(), "Red"), houses),
    (membero, ("Spaniard", "Dog", var(), var(), var()), houses),
    (membero, (var(), var(), "Coffee", var(), "Green"), houses),
    (membero, ("Ukrainian", var(), "Tea", var(), var()), houses),
    (leftof, (var(),var(),var(),var(),"Ivory"), (var(),var(),var(),var(),"Green"), houses),
    (membero, (var(), "Snails", var(), "Old Gold", var()), houses),
    (membero, (var(), var(), var(), "Kools", "Yellow"), houses),
    (eq, (var(),var(),(var(),var(),"Milk",var(),var()),var(),var()), houses),
    (eq, (("Norwegian",var(),var(),var(),var()),var(),var(),var(),var()), houses),
    (nextto, (var(),var(),var(),"Chesterfields",var()), (var(),"Fox",var(),var(),var()), houses),
    (nextto, (var(),var(),var(),"Kools",var()), (var(),"Horse",var(),var(),var()), houses),
    (membero, (var(), var(), var(), "Kools", "Yellow"), houses),
    (membero, ("Japanese", var(), var(), "Parliaments", var()), houses),
    (membero, (var(), var(), "Orange Juice", "Lucky Strike", var()), houses),
    (nextto, ("Norwegian",var(),var(),var(),var()), (var(),var(),var(),var(),"Blue"), houses),
    (membero, (var(), var(), "Water", var(), var()), houses),
    (membero, (var(), "Zebra", var(), var(), var()), houses)
)

solution = run(0, houses, rules)
for s in solution:
    for i in s:
        print(i)

('Norwegian', 'Fox', 'Water', 'Kools', 'Yellow')
('Ukrainian', 'Horse', 'Tea', 'Chesterfields', 'Blue')
('Englishman', 'Snails', 'Milk', 'Old Gold', 'Red')
('Spaniard', 'Dog', 'Orange Juice', 'Lucky Strike', 'Ivory')
('Japanese', 'Zebra', 'Coffee', 'Parliaments', 'Green')
